In [78]:
import pandas as pd
import numpy as np
#allgenres = pd.read_excel(r'C:\Users\Nathan\Documents\music recommendation system\allgenres.xlsx',header=None)
song_df = pd.read_csv(r'C:\Users\Nathan\Documents\music recommendation system\songs_subset.csv',header=None)
#genres1 = pd.read_excel(r'C:\Users\Nathan\Documents\music recommendation system\clean_genres.xlsx',header=None)
#genres2 = pd.read_excel(r'C:\Users\Nathan\Documents\music recommendation system\clean_genres2.xlsx',header=None)

### Grouping the genres obtained by the artists on the original dataset

In [79]:
df_genres = [] 
df_genres = [genres[j] for i in range(0,len(song_df)) for j in range(0,len(artists)) if song_df['artist_name'][i] == artists[j]]  

In [79]:
song_df.rename(columns={1: "user_id",2:"song_id",3:"listen_count",4:"title",5:"release",6:"artist_name",7:"year",8:"genre"},inplace=True)
song_df["song"] = song_df["title"] + " - " + song_df["artist_name"]

### Cleaning the songs without year or genre

In [80]:
song_df = song_df[song_df['year']!=0]

In [81]:
song_df = song_df.dropna()

In [82]:
song_df['genre']

0                                ['rumba', 'flamenco']
1                                              ['rap']
2                                       ['neo mellow']
3        ['permanent wave', 'rock', 'grunge', 'metal']
4                ['rock', 'musica aragonesa', 'latin']
                             ...                      
99994                       ['permanent wave', 'rock']
99995              ['indie', 'rock', 'permanent wave']
99997                                ['rock', 'blues']
99998                 ['rock', 'pop', 'punk', 'comic']
99999      ['rock', 'british invasion', 'mellow gold']
Name: genre, Length: 75051, dtype: object

In [83]:
genres = [None]*len(song_df)

In [84]:
song_df.reset_index(drop=True,inplace=True)

### Making List to String

In [85]:
import re
for i in range(0,len(song_df['genre'])):
        genre = ''
        str1 = re.findall("'(.*?)'",song_df['genre'][i])
        for j in range(0,len(str1)):
            genre = genre + '|' + str1[j]
        genre = genre[1:]
        genres[i] = genre

In [86]:
song_df['genre'] = genres

### grouping most popular songs

In [87]:
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
top_songs = song_grouped.sort_values(by=["listen_count"],ascending=False)
top_songs.reset_index(drop=True,inplace=True)

In [88]:
top_songs

,song,listen_count
0,Undo - Björk,367
1,You're The One - Dwight Yoakam,317
2,Secrets - OneRepublic,308
3,Revelry - Kings Of Leon,306
4,Fireflies - Charttraxx Karaoke,258
...,...,...
7525,Come Clarity - In Flames,1
7526,If This Isn't Love - Jennifer Hudson,1
7527,Punk (Radio Edit) - Ferry Corsten,1
7528,Coming Home - Fleetwood Mac,1


In [89]:
### df de song_df sem musicas repetidas
reduced_df = song_df.drop_duplicates(subset='song')
reduced_df.reset_index(drop=True,inplace=True)

In [93]:
###incluindo genero e ano em topsongs
topsongs_genres = []
topsongs_year = []
for i in range(0,len(top_songs)):
    for j in range(0,len(reduced_df)):
        if top_songs['song'][i] == reduced_df['song'][j]:
            topsongs_genres.append(reduced_df['genre'][j])
            topsongs_year.append(reduced_df['year'][j])

In [95]:
top_songs['genre'] = topsongs_genres
top_songs['year'] = topsongs_year

In [98]:
top_songs.to_excel(r'C:\Users\Nathan\Documents\music recommendation system\top_songs_complete.xlsx')

In [99]:
teste = pd.read_excel(r'C:\Users\Nathan\Documents\music recommendation system\top_songs_complete.xlsx')

In [102]:
top_songs['genre'][0]

'icelandic experimental|experimental vocal|permanent wave|pop|trip hop|electronica'

### Checking which is the most popular genres

In [118]:
all_genres = []

for i in range(0,len(top_songs)):
    if type(re.search('|',genres[i])) == re.Match:
        str1 = re.findall('(.*?)\|',top_songs['genre'][i])
        str2 = re.findall('\|([^|]*)$',top_songs['genre'][i])

        for j in range(0,len(str1)):
            all_genres.append(str1[j])
        if len(str2) == 1:
            all_genres.append(str2[0])

    else:
        all_genres.append(genres[i])

In [122]:
all_genres = pd.Series(all_genres)

In [124]:
top_genres = all_genres.value_counts()

In [131]:
top_25_genres = top_genres[0:25]

In [132]:
top_25_genres

rock                  3876
pop                   3158
indie                 1493
metal                  959
permanent wave         664
rap                    615
new rave               508
stomp and holler       478
folk                   473
electronica            462
neo mellow             428
grunge                 387
punk                   378
electronic             297
latin                  295
mellow gold            290
hardcore               288
urban contemporary     269
r&b                    254
jazz                   252
soul                   248
reggae                 216
melancholia            216
new wave               189
blues                  179
dtype: int64

In [149]:
type(top_25_genres.index)

pandas.core.indexes.base.Index

In [150]:
top25list = top_25_genres.index.tolist()

### Use only the 25 most popular genre, otherwise it will be categorized as 'other'

In [153]:

def check_top_25(string,top_25):
    check = 0
    for i in range(0,25):        
        if string == top_25[i]:
            check = 1
        
    if check == 0:
        string = 'other'
    
    return string
    
        

In [247]:
genres = [None]*len(top_songs)
for i in range(0,len(top_songs)):
    if type(re.search('|',top_songs['genre'][i])) == re.Match:
        str1 = re.findall('(.*?)\|',top_songs['genre'][i])
        for j in range(0,len(str1)):
            str1[j] = check_top_25(str1[j],top25list)  

        str2 = re.findall('\|([^|]*)$',top_songs['genre'][i])
        str2 = check_top_25(str2,top25list)
        str2 = [str2]

        
        genres[i] = str1 + str2
        genres[i] = list(dict.fromkeys(genres[i]))
    else:
        genres[i] = check_top_25(top_songs['genre'][i],top25list)

In [255]:
top_songs['genre'] = genres

In [255]:
top_songs = top_songs.explode('genre')

In [258]:
top_songs.to_excel(r'C:\Users\Nathan\Documents\music recommendation system\dashboard_topsongs.xlsx')